<a href="https://colab.research.google.com/github/singhnidh/Assessment-of-Fruit-Quality-using-Hyperspectral-Imaging-with-the-help-of-Deep-learning/blob/main/Alexnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
# import metric
from keras.metrics import sparse_categorical_crossentropy
# metric
from keras.metrics import binary_crossentropy
# optimization method (Stochastic Gradient Descent (SGD))
from keras.optimizers import SGD
# optimization method
from keras.optimizers import SGD
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import pathlib
from tensorflow.keras.optimizers import SGD, RMSprop, Adam
from zipfile import ZipFile
import cv2
import random
from tqdm import tqdm
from glob import glob
import tensorflow as tf
from tensorflow import keras
from keras.utils import np_utils
from keras_preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import tensorflow as tf
from keras.layers.core import Dense, Activation, Dropout, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn import metrics
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import matplotlib.image as mpimg
from numpy import *
from PIL import Image

In [ ]:
#DATADIR = 'D:/archivefruit/fruits-360_dataset/fruits-360/Training'
DATADIR ='E:\project\set project\SET 1\Fruit-classification-and-analysis-using-hyperspectral-images-and-deep-learning-main\data'
CATEGORIES = ["Apple Braeburn","Avocado","Banana","Blueberry","Guava","Kiwi","Orange"]
for category in CATEGORIES:
    path = os.path.join(DATADIR, category)
    for img in os.listdir(path):
        img_array = cv2.imread(os.path.join(path,img))
        #,cv2.IMREAD_GRAYSCALE
        plt.imshow(img_array)
        plt.show()
        break
   # break


In [ ]:
#building our training data
training_data = []

def create_training_data():
    for category in CATEGORIES:  # do fruits

        path = os.path.join(DATADIR,category)  # create path to fruits
        class_num = CATEGORIES.index(category)  # get the classification  (0 ,1,2,3,4,5,6) different number donate different category of fruit

        for img in tqdm(os.listdir(path)):  # iterate over each image per fruits
            try:
                img_array = cv2.imread(os.path.join(path,img) ,cv2.IMREAD_GRAYSCALE)  # convert to array
                new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))  # resize to normalize data size
                training_data.append([new_array, class_num])  # add this to our training_data
            except Exception as e:  # in the interest in keeping the output clean...
                pass

create_training_data()

image_count = len(training_data)

In [ ]:
print(image_count)

In [ ]:
#shuffle the dataset
random.shuffle(training_data)

In [ ]:
for sample in training_data[:10]:
    print(sample[1])

In [ ]:
IMG_SIZE = 100
batch_size = 128
img_height = 100
img_width = 100

In [ ]:
#Assigning labels and features
X =[]
y =[]
for features, label in training_data:
    X.append(features)
    y.append(label)
X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 1)

In [ ]:
#creating own model
#Normalising X and converting labels to categorical data
X = X.astype('float32')
X /= 255
Y = np_utils.to_categorical(y,7)
print(Y[100])
print(shape(Y))

In [ ]:
X = np.array(X)
print(type(X))

In [ ]:
y=np.array(y)
print(type(y))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 7)

In [ ]:
#define compare and train the CNN model
batch_size = 128
nb_classes =7
nb_epochs = 10
img_rows, img_columns = 100, 100
img_channel = 3
nb_filters = 32
nb_pool = 2
nb_conv = 3

In [ ]:
def Alexnet():

    # Initialize the model
    model = Sequential()

    # layer 1: convolutional layer + max-pooling layer
    model.add(Conv2D(filters = 96, kernel_size = (11,11), strides= 4, padding = 'valid', activation='relu', input_shape = (100,100,1)))
    model.add(MaxPooling2D(pool_size = (3,3), strides = 2))

    # layer 2: convolutional layer + max-pooling layer
    model.add(Conv2D(filters = 256, kernel_size = (5,5), padding = 'same', activation = 'relu'))
    model.add(MaxPooling2D(pool_size = (3,3), strides = 2))

    # layers 3-5: three convolutional layers + 1 max-pooling layer
    model.add(Conv2D(filters = 384, kernel_size = (3,3), padding = 'same', activation = 'relu'))
    model.add(Conv2D(filters = 384, kernel_size = (3,3), padding = 'same', activation = 'relu'))
    model.add(Conv2D(filters = 256, kernel_size = (3,3), padding = 'same', activation = 'relu'))
    model.add(MaxPooling2D(pool_size = (3,3), strides = 2))

    # layers 6 - 8: two fully connected hidden layers and one fully connected output layer
    model.add(Dense(4096, activation = 'relu',input_dim=4))
    model.add(Flatten())
    model.add(Dropout(0.5))
    model.add(Dense(4096, activation = 'relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1000, activation = 'softmax'))

    # compile the model with a loss funciton, a metric and optimizer method for estimating the loss function
    opt = SGD(lr = 0.1)
    model.compile(loss = sparse_categorical_crossentropy,optimizer = opt,metrics = ['accuracy'])
    return model


In [ ]:
Alexnet_model = Alexnet()
Alexnet_model.summary()

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint

def train_model(model, X_train, y_train, X_test, y_test, epochs, batch_size):
  # Data generator
  datagen = ImageDataGenerator(rotation_range = 5, width_shift_range = 0.1, height_shift_range = 0.1, horizontal_flip = True)
  # iteration on the training set
  it_train = datagen.flow(X_train, y_train, batch_size = batch_size)
  # path to save the checkpoint
  path_cp = os.getcwd() + '/' + 'weights_.hdf5'
  checkpoint_ = ModelCheckpoint(path_cp, monitor = 'loss', save_best_only = True, mode = 'auto')
  steps = X_train.shape[0]//batch_size
  # Fitting the model
  history = model.fit(it_train, epochs = epochs, steps_per_epoch = steps, validation_data = (X_test, y_test), verbose = 1, callbacks = checkpoint_)
  # Evaluating the model
  _, acc = model.evaluate(X_test, y_test, verbose = 1)
  print('%.3f' % (acc * 100.0))

  return history, acc

train_history, acc = train_model(Alexnet_model, X_train, y_train, X_test, y_test, batch_size = 32, epochs = 10)

In [ ]:
import pandas as pd

# convert the history.history to a pandas DataFrame:
train_hist_df = pd.DataFrame(train_history.history)

train_dict = {'train_hist': train_hist_df, 'acc': acc}
np.save('train_dict.npy', train_dict)

In [ ]:
Alexnet_model.save('Alexnet_model.h5')

In [ ]:
def plot_accuracy(history):
  plt.figure(figsize = (5,5))
  plt.plot(history .history['accuracy'], color = 'blue', label = 'train')
  plt.plot(history.history['val_accuracy'], color = 'orange', label = 'val')
  plt.legend()
  plt.title('Accuracy')
  plt.show()

plot_accuracy(train_history)

In [ ]:
#Accuracy and Score of model
score = Alexnet_model.evaluate(X_test, y_test, verbose = 1 )
print("Test Score: ", score[0])
print("Test accuracy: ", score[1]*100,'%')

In [ ]:
#observing its classification report and confusion matrix
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
#predict
y_pred=Alexnet_model.predict(X_test)
y_pred=np.argmax(y_pred,axis=1)
print(classification_report(y_pred,y_test))

In [ ]:
print(confusion_matrix(y_pred,y_test))

In [ ]:
#get classification report
print(classification_report(y_pred,y_test))

In [ ]:
#get confusion matrix
print(confusion_matrix(y_pred,y_test))

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

# Reverse the true and the predicted labels into the original binary
# labels (0 and 1) to compute the confusion matrix
y_test_labels = [np.argmax(vect) for vect in y_test]
y_test_pred_labels = [np.argmax(vect) for vect in y_test_labels]

conf_mat = confusion_matrix(y_test_labels, y_test_pred_labels)

plt.figure(figsize = (10,8))
sns.heatmap(conf_mat, linewidths = 0.1, cmap = 'Greens', linecolor = 'gray',
            fmt = '.1f', annot = True)
plt.xlabel('Predicted classes', fontsize = 20)
plt.ylabel('True classes', fontsize = 20)

In [ ]:
plt.figure(figsize = (15,7))
for i in np.arange(7):
  # take randomly an indice
  ind = random.randint(0, len(X_test))
  img = X_test[ind]
  # reshape the image
  img_rs = img.reshape(1,100,100,1)
  # predict the label of img
  y_pred = Alexnet_model.predict(img_rs)
  # determine the corresponding category
  predicted_cate = category[np.argmax(y_pred)]
  plt.subplot(240+1+i)
  plt.imshow(img)
  plt.title('predicted: ' + str(predicted_cate))